<a href="https://colab.research.google.com/github/Hashara/CS4622-Machine-Learning-Project/blob/main/CS4622_Machine_Learning_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libararies

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import statsmodels.api as sm
from sklearn import metrics

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import OrdinalEncoder

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Read CSVs

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df_train_val = pd.read_csv('/content/drive/MyDrive/sem 7/ML - sem 7/Ml project/dataset/training_set_values.csv')
df_test_val = pd.read_csv('/content/drive/MyDrive/sem 7/ML - sem 7/Ml project/dataset/test_set_values.csv')
df_train_label = pd.read_csv('/content/drive/MyDrive/sem 7/ML - sem 7/Ml project/dataset/training_set_labels.csv')
df_submission = pd.read_csv('/content/drive/MyDrive/sem 7/ML - sem 7/Ml project/dataset/SubmissionFormat.csv')

In [5]:
df_train_val

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,10.0,2013-05-03,Germany Republi,1210,CES,37.169807,-3.253847,Area Three Namba 27,0,Pangani,Kiduruni,Kilimanjaro,3,5,Hai,Masama Magharibi,125,True,GeoData Consultants Ltd,Water Board,Losaa Kia water supply,True,1999,gravity,gravity,gravity,water board,user-group,pay per bucket,per bucket,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
59396,27263,4700.0,2011-05-07,Cefa-njombe,1212,Cefa,35.249991,-9.070629,Kwa Yahona Kuvala,0,Rufiji,Igumbilo,Iringa,11,4,Njombe,Ikondo,56,True,GeoData Consultants Ltd,VWC,Ikondo electrical water sch,True,1996,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe
59397,37057,0.0,2011-04-11,NaN,0,NaN,34.017087,-8.750434,Mashine,0,Rufiji,Madungulu,Mbeya,12,7,Mbarali,Chimala,0,True,GeoData Consultants Ltd,VWC,NaN,False,0,swn 80,swn 80,handpump,vwc,user-group,pay monthly,monthly,fluoride,fluoride,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump
59398,31282,0.0,2011-03-08,Malec,0,Musa,35.861315,-6.378573,Mshoro,0,Rufiji,Mwinyi,Dodoma,1,4,Chamwino,Mvumi Makulu,0,True,GeoData Consultants Ltd,VWC,NaN,True,0,nira/tanira,nira/tanira,handpump,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump


In [6]:
df_train_label

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional
...,...,...
59395,60739,functional
59396,27263,functional
59397,37057,functional
59398,31282,functional


# Feature engineering

## Missing Values

In [7]:
cols_with_missing = [col for col in df_train_val.columns
                     if df_train_val[col].isnull().any()]

In [8]:
cols_with_missing

['funder',
 'installer',
 'subvillage',
 'public_meeting',
 'scheme_management',
 'scheme_name',
 'permit']

In [9]:
cols_with_missing_test = [col for col in df_test_val.columns
                     if df_test_val[col].isnull().any()]

In [10]:
# drop missing values
df_train_val = df_train_val.drop(columns = cols_with_missing)
df_test_val = df_test_val.drop(columns = cols_with_missing)

## Exploring data

In [11]:
df_train_val.columns

Index(['id', 'amount_tsh', 'date_recorded', 'gps_height', 'longitude',
       'latitude', 'wpt_name', 'num_private', 'basin', 'region', 'region_code',
       'district_code', 'lga', 'ward', 'population', 'recorded_by',
       'construction_year', 'extraction_type', 'extraction_type_group',
       'extraction_type_class', 'management', 'management_group', 'payment',
       'payment_type', 'water_quality', 'quality_group', 'quantity',
       'quantity_group', 'source', 'source_type', 'source_class',
       'waterpoint_type', 'waterpoint_type_group'],
      dtype='object')

In [12]:
df_train_val.head()

,id,amount_tsh,date_recorded,gps_height,longitude,latitude,wpt_name,num_private,basin,region,region_code,district_code,lga,ward,population,recorded_by,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,1390,34.938093,-9.856322,none,0,Lake Nyasa,Iringa,11,5,Ludewa,Mundindi,109,GeoData Consultants Ltd,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,1399,34.698766,-2.147466,Zahanati,0,Lake Victoria,Mara,20,2,Serengeti,Natta,280,GeoData Consultants Ltd,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,686,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Manyara,21,4,Simanjiro,Ngorika,250,GeoData Consultants Ltd,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,263,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mtwara,90,63,Nanyumbu,Nanyumbu,58,GeoData Consultants Ltd,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,0,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kagera,18,1,Karagwe,Nyakasimbi,0,GeoData Consultants Ltd,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [13]:
df_train_val.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59400.000000,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475
std,21453.128371,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18519.750000,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37061.500000,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55656.500000,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


In [14]:
df_train_val.corr()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
id,1.000000,-0.005321,-0.004692,-0.001348,0.001718,-0.002629,-0.003028,-0.003044,-0.002813,-0.002082
amount_tsh,-0.005321,1.000000,0.076650,0.022134,-0.052670,0.002944,-0.026813,-0.023599,0.016288,0.067915
gps_height,-0.004692,0.076650,1.000000,0.149155,-0.035751,0.007237,-0.183521,-0.171233,0.135003,0.658727
longitude,-0.001348,0.022134,0.149155,1.000000,-0.425802,0.023873,0.034197,0.151398,0.086590,0.396732
latitude,0.001718,-0.052670,-0.035751,-0.425802,1.000000,0.006837,-0.221018,-0.201020,-0.022152,-0.245278
num_private,-0.002629,0.002944,0.007237,0.023873,0.006837,1.000000,-0.020377,-0.004478,0.003818,0.026056
region_code,-0.003028,-0.026813,-0.183521,0.034197,-0.221018,-0.020377,1.000000,0.678602,0.094088,0.031724
district_code,-0.003044,-0.023599,-0.171233,0.151398,-0.201020,-0.004478,0.678602,1.000000,0.061831,0.048315
population,-0.002813,0.016288,0.135003,0.086590,-0.022152,0.003818,0.094088,0.061831,1.000000,0.260910
construction_year,-0.002082,0.067915,0.658727,0.396732,-0.245278,0.026056,0.031724,0.048315,0.260910,1.000000


## Adding new features

In [15]:
df_train_val.head()

,id,amount_tsh,date_recorded,gps_height,longitude,latitude,wpt_name,num_private,basin,region,region_code,district_code,lga,ward,population,recorded_by,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,1390,34.938093,-9.856322,none,0,Lake Nyasa,Iringa,11,5,Ludewa,Mundindi,109,GeoData Consultants Ltd,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,1399,34.698766,-2.147466,Zahanati,0,Lake Victoria,Mara,20,2,Serengeti,Natta,280,GeoData Consultants Ltd,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,686,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Manyara,21,4,Simanjiro,Ngorika,250,GeoData Consultants Ltd,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,263,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mtwara,90,63,Nanyumbu,Nanyumbu,58,GeoData Consultants Ltd,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,0,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kagera,18,1,Karagwe,Nyakasimbi,0,GeoData Consultants Ltd,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


### Number of years

In [16]:
df_train_val["year_recorded"] = df_train_val["date_recorded"].astype(str).str[0:4].astype(int)
df_test_val["year_recorded"] = df_test_val["date_recorded"].astype(str).str[0:4].astype(int)

In [17]:
df_train_val["number_of_years"] = df_train_val["year_recorded"] - df_train_val["construction_year"]
df_test_val["number_of_years"] = df_test_val["year_recorded"] - df_test_val["construction_year"]

In [18]:
df_train_val["number_of_years"].value_counts()

 2011    13343
 2012     5142
 3        2740
 1        2303
 2013     2210
         ...  
-1           1
-3           1
-2           1
 2002        1
-7           1
Name: number_of_years, Length: 65, dtype: int64

In [19]:
x = pd.DataFrame()
x["number_of_years"] = df_train_val["number_of_years"]
x = x.drop(x[x.number_of_years > 2000].index)
x["number_of_years"].describe()

count    38691.000000
mean        15.355742
std         12.492673
min         -7.000000
25%          5.000000
50%         13.000000
75%         25.000000
max         53.000000
Name: number_of_years, dtype: float64

In [20]:
df_train_val.loc[df_train_val.number_of_years > 2000, 'number_of_years'] = 15
df_test_val.loc[df_train_val.number_of_years > 2000, 'number_of_years'] = 15

In [21]:
df_train_val.head()

,id,amount_tsh,date_recorded,gps_height,longitude,latitude,wpt_name,num_private,basin,region,region_code,district_code,lga,ward,population,recorded_by,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,year_recorded,number_of_years
0,69572,6000.0,2011-03-14,1390,34.938093,-9.856322,none,0,Lake Nyasa,Iringa,11,5,Ludewa,Mundindi,109,GeoData Consultants Ltd,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,2011,12
1,8776,0.0,2013-03-06,1399,34.698766,-2.147466,Zahanati,0,Lake Victoria,Mara,20,2,Serengeti,Natta,280,GeoData Consultants Ltd,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,2013,3
2,34310,25.0,2013-02-25,686,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Manyara,21,4,Simanjiro,Ngorika,250,GeoData Consultants Ltd,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,2013,4
3,67743,0.0,2013-01-28,263,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mtwara,90,63,Nanyumbu,Nanyumbu,58,GeoData Consultants Ltd,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,2013,27
4,19728,0.0,2011-07-13,0,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kagera,18,1,Karagwe,Nyakasimbi,0,GeoData Consultants Ltd,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,2011,15


In [22]:
df_train_val["status_group"] = df_train_label["status_group"]

## Encoding

In [117]:
# object_cols = [ 'basin', 'lga', 'ward','extraction_type','extraction_type_group',
#                'extraction_type_class','management', 'management_group', 'payment',	'payment_type',
#                'water_quality',	'quality_group',	'quantity',	'quantity_group',	'source',	'source_type',
#                'source_class',	'waterpoint_type',	'waterpoint_type_group']
object_cols = ['payment_type', 'source_type', 'quantity_group','quality_group', 'management', 'extraction_type_group']
# , 'basin', 'lga', 'ward','extraction_type','extraction_type_group',
#                'extraction_type_class','management', 'management_group', 'payment',	'payment_type',
#                'water_quality',	'quality_group',	'quantity',	'quantity_group',	'source',	'source_type',
#                'source_class',	'waterpoint_type',	'waterpoint_type_group']

In [118]:
# label_X_train = df_train_val.copy()
# label_X_test = df_test_val.copy()
df_train_val = label_X_train.copy()
df_test_val = label_X_test.copy()

In [119]:
# Apply ordinal encoder to each column with categorical data
ordinal_encoder = OrdinalEncoder()
df_train_val[object_cols] = ordinal_encoder.fit_transform(df_train_val[object_cols])
df_test_val[object_cols] = ordinal_encoder.transform(df_test_val[object_cols])

In [120]:
df_test_val.head()

,id,amount_tsh,date_recorded,gps_height,longitude,latitude,wpt_name,num_private,basin,region,region_code,district_code,lga,ward,population,recorded_by,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,year_recorded,number_of_years
0,50785,0.0,2013-02-04,1996,35.290799,-4.059696,Dinamu Secondary School,0,Internal,Manyara,21,3,63.0,Bashay,321,GeoData Consultants Ltd,2012,other,6.0,other,3.0,parastatal,never pay,2.0,soft,2.0,seasonal,3.0,rainwater harvesting,3.0,surface,other,other,2013,1
1,51630,0.0,2013-02-04,1569,36.656709,-3.309214,Kimnyak,0,Pangani,Arusha,2,2,0.0,Kimnyaki,300,GeoData Consultants Ltd,2000,gravity,1.0,gravity,7.0,user-group,never pay,2.0,soft,2.0,insufficient,2.0,spring,6.0,groundwater,communal standpipe,communal standpipe,2013,13
2,17168,0.0,2013-02-01,1567,34.767863,-5.004344,Puma Secondary,0,Internal,Singida,13,2,109.0,Puma,500,GeoData Consultants Ltd,2010,other,6.0,other,7.0,user-group,never pay,2.0,soft,2.0,insufficient,2.0,rainwater harvesting,3.0,surface,other,other,2013,3
3,45559,0.0,2013-01-22,267,38.058046,-9.418672,Kwa Mzee Pange,0,Ruvuma / Southern Coast,Lindi,80,43,49.0,Mkutano,250,GeoData Consultants Ltd,1987,other,6.0,other,7.0,user-group,unknown,6.0,soft,2.0,dry,0.0,shallow well,5.0,groundwater,other,other,2013,26
4,49871,500.0,2013-03-27,1260,35.006123,-10.950412,Kwa Mzee Turuka,0,Ruvuma / Southern Coast,Ruvuma,10,3,61.0,Mbinga Urban,60,GeoData Consultants Ltd,2000,gravity,1.0,gravity,9.0,user-group,pay monthly,1.0,soft,2.0,enough,1.0,spring,6.0,groundwater,communal standpipe,communal standpipe,2013,13


In [121]:
label_X_train.corr()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,payment_type,source_type,year_recorded,number_of_years
id,1.000000,-0.005321,-0.004692,-0.001348,0.001718,-0.002629,-0.003028,-0.003044,-0.002813,-0.002082,-0.000443,-0.003094,0.002128,0.003732
amount_tsh,-0.005321,1.000000,0.076650,0.022134,-0.052670,0.002944,-0.026813,-0.023599,0.016288,0.067915,-0.118563,-0.000711,-0.004743,-0.007459
gps_height,-0.004692,0.076650,1.000000,0.149155,-0.035751,0.007237,-0.183521,-0.171233,0.135003,0.658727,-0.097431,0.124323,0.318373,0.023496
longitude,-0.001348,0.022134,0.149155,1.000000,-0.425802,0.023873,0.034197,0.151398,0.086590,0.396732,-0.007441,-0.047479,0.010927,-0.004398
latitude,0.001718,-0.052670,-0.035751,-0.425802,1.000000,0.006837,-0.221018,-0.201020,-0.022152,-0.245278,0.145438,0.064626,0.160288,-0.017007
num_private,-0.002629,0.002944,0.007237,0.023873,0.006837,1.000000,-0.020377,-0.004478,0.003818,0.026056,0.008963,-0.010613,-0.013816,0.001487
region_code,-0.003028,-0.026813,-0.183521,0.034197,-0.221018,-0.020377,1.000000,0.678602,0.094088,0.031724,0.140829,-0.141470,0.165532,0.022380
district_code,-0.003044,-0.023599,-0.171233,0.151398,-0.201020,-0.004478,0.678602,1.000000,0.061831,0.048315,0.093541,-0.068146,0.015315,-0.003732
population,-0.002813,0.016288,0.135003,0.086590,-0.022152,0.003818,0.094088,0.061831,1.000000,0.260910,0.034478,-0.090925,0.131747,-0.023054
construction_year,-0.002082,0.067915,0.658727,0.396732,-0.245278,0.026056,0.031724,0.048315,0.260910,1.000000,-0.016861,-0.025503,0.354792,0.006267


## Ploting data

In [ ]:
plt.figure(figsize=(20,20))

In [ ]:
sns.relplot(x="id",y="number_of_years",hue="status_group",data=df_train_val);

In [ ]:
sns.relplot(x="id",y="amount_tsh",hue="status_group",data=df_train_val);

In [ ]:
sns.relplot(x="id",y="gps_height",hue="status_group",data=df_train_val);

In [ ]:
sns.relplot(x="id",y="num_private",hue="status_group",data=df_train_val);

In [ ]:
sns.relplot(x="id",y="basin",hue="status_group",data=df_train_val);

In [ ]:
sns.relplot(x="id",y="region",hue="status_group",data=df_train_val);

In [ ]:
sns.relplot(x="id",y="region_code",hue="status_group",data=df_train_val);

In [ ]:
sns.relplot(x="id",y="district_code",hue="status_group",data=df_train_val);

In [ ]:
df_train_val.status_group.value_counts().plot.barh()

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(x='status_group', hue='number_of_years', data= df_train_val)

# Clean dataset

In [113]:
df_train_val.columns

Index(['id', 'amount_tsh', 'date_recorded', 'gps_height', 'longitude',
       'latitude', 'wpt_name', 'num_private', 'basin', 'region', 'region_code',
       'district_code', 'lga', 'ward', 'population', 'recorded_by',
       'construction_year', 'extraction_type', 'extraction_type_group',
       'extraction_type_class', 'management', 'management_group', 'payment',
       'payment_type', 'water_quality', 'quality_group', 'quantity',
       'quantity_group', 'source', 'source_type', 'source_class',
       'waterpoint_type', 'waterpoint_type_group', 'year_recorded',
       'number_of_years', 'status_group'],
      dtype='object')

In [122]:
df_train_clean = df_train_val.drop(columns=['id', 'amount_tsh', 'date_recorded', 'gps_height', 'longitude',
       'latitude', 'wpt_name', 'basin', 'region', 
       'lga', 'ward', 'population', 'recorded_by',
       'construction_year', 'extraction_type',
       'extraction_type_class',  'management_group', 'payment',
       'water_quality',  'quantity',
       'source', 'source_class',
       'waterpoint_type', 'waterpoint_type_group', 'year_recorded',
       'status_group'])
df_test_clean = df_test_val.drop(columns=['id', 'amount_tsh', 'date_recorded', 'gps_height', 'longitude',
       'latitude', 'wpt_name',  'basin', 'region', 
       'lga', 'ward', 'population', 'recorded_by',
       'construction_year', 'extraction_type', 
       'extraction_type_class', 'management_group', 'payment',
       'water_quality', 'quantity',
        'source',  'source_class',
       'waterpoint_type', 'waterpoint_type_group', 'year_recorded'])
df_train_label_clean = df_train_val["status_group"]

In [123]:
df_test_clean.head()

,num_private,region_code,district_code,lga,extraction_type_group,management,payment_type,quality_group,quantity_group,source_type,number_of_years
0,0,21,3,63.0,6.0,3.0,2.0,2.0,3.0,3.0,1
1,0,2,2,0.0,1.0,7.0,2.0,2.0,2.0,6.0,13
2,0,13,2,109.0,6.0,7.0,2.0,2.0,2.0,3.0,3
3,0,80,43,49.0,6.0,7.0,6.0,2.0,0.0,5.0,26
4,0,10,3,61.0,1.0,9.0,1.0,2.0,1.0,6.0,13


In [124]:
X_train, X_test, y_train, y_test = train_test_split(df_train_clean, df_train_label_clean, test_size=0.2, random_state=10, shuffle=True)

# Model

In [125]:
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [126]:
predictions = model.predict(X_test)

In [127]:
logreg_score_train = model.score(X_train,y_train)
print("Train Prediction Score",logreg_score_train*100)
logreg_score_test = accuracy_score(y_test,predictions)
print("Test Prediction ",logreg_score_test*100)

Train Prediction Score 71.43728956228956
Test Prediction  71.05218855218855


# Write to CSV

In [75]:
df_submission["status_group"] = model.predict(df_test_clean)

In [ ]:
# df_submission["status_group"].replace({"predicted label": "functional"}, inplace=True)

In [ ]:
df_submission.to_csv("SubmissionFormat.csv", index= False)